In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from typing import List
import json

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('******')

if api_key and api_key.startswith('****')and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key")

MODEL = 'gpt-4o-mini'
openai = OpenAI()


API key looks good so far


In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url= url
        response = requests.get(url, headers = headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "no title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
         self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"












In [ ]:
ed = Website("any site url")
ed.links

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/openbmb/MiniCPM-V-4_5',
 '/tencent/Hunyuan-MT-7B',
 '/meituan-longcat/LongCat-Flash-Chat',
 '/Qwen/Qwen-Image-Edit',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/apple/fastvlm-webgpu',
 '/spaces/multimodalart/Qwen-Image-Edit-Fast',
 '/spaces/multimodalart/nano-banana',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/openai/healthbench',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/facebook/recycling_the_web',
 '/datasets/nvidia/Nemotron-CC-Math-v1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/

In [ ]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
microsoft/VibeVoice-1.5B
Updated
about 23 hours ago
•
107k
•
1.23k
openbmb/MiniCPM-V-4_5
Updated
about 6 hours ago
•
11.3k
•
812
tencent/Hunyuan-MT-7B
Updated
about 3 hours ago
•
18
•
322
meituan-longcat/LongCat-Flash-Chat
Updated
2 days ago
•
218
•
312
Qwen/Qwen-Image-Edit
Updated
8 days ago
•
77.6k
•
1.61k
Browse 1M+ models
Spaces
Running
12.6k
12.6k
De

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\nabout 23 hours ago\n•\n107k\n•\n1.23k\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 6 hours ago\n•\n11.3k\n•\n812\ntencent/Hunyuan-MT-7B\nUpdated\nabout 3 hours ago\n•\n18\n•\n322\nmeituan-longcat/LongCat-Flash-Chat\nUpdated\n2 days ago\n•\n218\n•\n314\nQwen/Qwen-Image-Edit\nUpdated\n8 days ago\n•\n77.6k\n•\n1.61k\nBrowse 1M+ models\nSpaces\nRunning\n12.6k\n12.6

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


```markdown
# Hugging Face Brochure

## About Us
Welcome to **Hugging Face**, a thriving AI community that is shaping the future of machine learning. Our platform enables developers and researchers to collaborate on **models**, **datasets**, and **applications**, resulting in innovative AI solutions. With over **1 million models** and **250,000 datasets**, we provide the tools necessary to explore, build, and share within the AI ecosystem.

---

## Our Mission
At Hugging Face, our mission is to create an open-source foundation for machine learning tooling, supporting growth in various modalities such as text, image, video, audio, and 3D. We empower our community to leverage our resources to accelerate machine learning innovation.

---

## Company Culture
Hugging Face prides itself on a **collaborative and inclusive culture**. As a member of our community, you'll engage with like-minded individuals who are passionate about AI technology. We promote open communication and knowledge sharing, encouraging a spirit of support among peers. Our commitment to diversity and inclusion enhances our collaborative efforts, making us stronger together.

---

## Customers & Partnerships
We are proud to serve more than **50,000 organizations** across various industries, including notable names like:
- **Google**
- **Amazon**
- **Meta**
- **Microsoft**

These partnerships allow us to collect a wealth of models and datasets, enriching our ecosystem and ensuring that we remain at the forefront of AI development.

---

## Careers at Hugging Face
Join us at Hugging Face! We are always on the lookout for talented individuals to join our team. If you're passionate about AI and want to make a meaningful impact, we would love to hear from you. Our job openings are diverse, offering opportunities in engineering, data science, community engagement, and more. Discover a rewarding career path in a collaborative environment that fosters personal and professional growth.

---

## Explore More
- **Models**: Search through over **1 million models** and discover the latest AI innovations.
- **Datasets**: Access a wide range of over **250,000 datasets** to fuel your next project.
- **Spaces**: Engage with various applications and services, collaborating with the community to create groundbreaking solutions.

Visit our website to learn more, sign up, or explore our resources today!

[Visit Hugging Face](https://huggingface.co)  
[Sign Up](https://huggingface.co/join)  

---

Together, let’s build the future of AI!
```


In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}



# Hugging Face Brochure

## Welcome to Hugging Face
**The AI Community Building the Future**  
Hugging Face is at the forefront of the AI and machine learning revolution, providing a collaborative platform where innovators can create, share, and explore state-of-the-art AI models, datasets, and applications. With over one million models and 250,000 datasets available, our goal is to accelerate machine learning development and empower users worldwide.

---

## Our Offerings
- **Models**: Browse a vast array of AI models including text, image, and video processing capabilities.
- **Datasets**: Access and contribute to a diverse range of datasets to refine and enhance ML projects.
- **Spaces**: Engage with applications crafted by the community, from video generation to real-time captioning.
- **Enterprise Solutions**: Harness enterprise-grade security, dedicated support, and advanced deployment options for teams.

---

## Community and Collaboration
At Hugging Face, community is our backbone. More than 50,000 organizations, including leading tech companies like Google, Microsoft, and Amazon, are collaborating within the Hugging Face ecosystem. We facilitate meaningful contributions from users who can share models and datasets, helping others learn and innovate. 

### Stay Connected:
- **Follow Us**: Join our journey via our [GitHub](https://github.com/huggingface) and social media platforms like [Twitter](https://twitter.com/huggingface) and [Discord](https://discord.gg/huggingface).
- **Documentation**: Check out our comprehensive [Docs](https://huggingface.co/docs) to get started or deepen your knowledge.

---

## Company Culture
Hugging Face thrives on a culture of openness and inclusivity. We believe in the potential of every developer and researcher, encouraging our team and community members to express their artistic and intellectual prowess. Our open-source ethos fosters collaboration, trust, and mutual growth, making AI and ML accessible to all.

---

## Careers at Hugging Face
We are constantly searching for passionate, talented individuals to join our mission. If you're excited to work in a dynamic environment filled with creativity and innovation, explore our current [Job Openings](https://huggingface.co/jobs). 

### Why Join Us?
- Be part of a mission-driven company focused on democratizing AI.
- Collaborate with industry leaders and creative minds.
- Enjoy a flexible work environment with a commitment to personal development.

---

## Join Us!
Whether you're an AI enthusiast looking to enhance your skills, a company searching for advanced ML solutions, or a professional eager to join an innovative team, Hugging Face welcomes you to be part of our vibrant community!

*Explore our platform today at [huggingface.co](https://huggingface.co)*

